# Satellite Configuration

[Satellites](../api_reference/sats/index.rst) are the basic unit of agent in the 
environment. Four things must be specified in subclasses of `Satellite`:

* The `observation_spec`, which defines the satellite's [observation](../api_reference/obs/index.rst).
* The `action_spec`, which defines the satellite's [actions](../api_reference/act/index.rst).
* The `dyn_type`,  which selects the underlying [dynamics model](../api_reference/sim/dyn.rst) used in simulation.
* The `fsw_type`,  which selects the underlying [flight software model](../api_reference/sim/fsw.rst).

A very simple satellite is defined below:

In [ ]:
from bsk_rl import sats, act, obs, scene, data, SatelliteTasking
from bsk_rl.sim import dyn, fsw
import numpy as np

from Basilisk.architecture import bskLogging
bskLogging.setDefaultLogLevel(bskLogging.BSK_WARNING)


class SimpleSatellite(sats.Satellite):
    observation_spec = [obs.Time()]  # Passed as list of instantiated classes
    action_spec = [act.Drift()]
    dyn_type = dyn.BasicDynamicsModel  # Passed as a type
    fsw_type = fsw.BasicFSWModel

## Setting Satellite Parameters

Without instantiating the satellite, parameters that can be set in the various models
can be inspected.

In [ ]:
SimpleSatellite.default_sat_args()

These parameters can be overriden when instantiating the satellite through the `sat_args`
argument. 

In [ ]:
sat = SimpleSatellite(
    name="SimpleSat-1",
    sat_args=dict(
        mass=300,  # Setting a constant value
        dragCoeff=lambda: np.random.uniform(2.0, 2.4),  # Setting a randomized value
    ),
)


Each time the simulation is reset, all of the function-based randomizers are called.

In [ ]:
sat.generate_sat_args()  # Called by the environment on reset()
sat.sat_args

As a result, each episode will have different randomized parameters:

In [ ]:
for _ in range(3):
    sat.generate_sat_args()  # Called by the environment on reset()
    print("New value of dragCoeff:", sat.sat_args["dragCoeff"])

## The Observation Specification

A variety of observation elements are available for satellites. Full documentation
can be [found here](../api_reference/obs/index.rst), but some commonly used elements
are explored below.

<div class="alert alert-info">

**Info:** In these examples, `obs_type=dict` is passed to the `Satellite` constructor
so that the observation is human readable. While some RL libraries support dictionary-based
observations, the default return type - the numpy array format - is more typically used.

</div>


### Satellite Properties

The most common type of observations is introspective; i.e. what is my current state?
Any `@property` in the `dyn_type` or `fsw_type` of the satellite can be accessed using
SatProperties.

In [ ]:
class SatPropsSatellite(sats.Satellite):
    observation_spec = [
        obs.SatProperties(
            # At a minimum, specify the property to observe
            dict(prop="wheel_speeds"),
            # You can specify the module to use for the observation, but it is not necessary
            # if only one module has for the property
            dict(prop="battery_charge_fraction", module="dynamics"), 
            # Properties can be normalized by some constant. This is generally desirable
            # for RL algorithms to keep values around [-1, 1].
            dict(prop="r_BN_P", norm=7e6),
        )
    ]
    action_spec = [act.Drift()]
    dyn_type = dyn.BasicDynamicsModel
    fsw_type = fsw.BasicFSWModel

env = SatelliteTasking(
    satellite=SatPropsSatellite("PropSat-1", {}, obs_type=dict),
    log_level="CRITICAL",
)
observation, _ = env.reset()
observation

In some cases, you may want to access a bespoke property that is not natively implemented
in a model. To do that, simply extend the model with your desired property.

In [ ]:
class BespokeFSWModel(fsw.BasicFSWModel):
    @property
    def meaning_of_life(self):
        return 42
    
class BespokeSatPropsSatellite(sats.Satellite):
    observation_spec = [
        obs.SatProperties(dict(prop="meaning_of_life"))
    ]
    action_spec = [act.Drift()]
    dyn_type = dyn.BasicDynamicsModel
    fsw_type = BespokeFSWModel

env = SatelliteTasking(
    satellite=BespokeSatPropsSatellite("BespokeSat-1", {}, obs_type=dict),
    log_level="CRITICAL",
)
observation, _ = env.reset()
observation

Alternatively, define the property with a function that takes the satellite object as an argument.

In [ ]:
class CustomSatPropsSatellite(sats.Satellite):
    observation_spec = [
        obs.SatProperties(dict(prop="meaning_of_life", fn=lambda sat: 42))
    ]
    action_spec = [act.Drift()]
    dyn_type = dyn.BasicDynamicsModel
    fsw_type = fsw.BasicFSWModel

env = SatelliteTasking(
    satellite=CustomSatPropsSatellite("BespokeSat-1", {}, obs_type=dict),
    log_level="CRITICAL",
)
observation, _ = env.reset()
observation

### Opportunity Properties
Another common input to the observation is information about upcoming locations that 
are being accessed by the satellite. Currently, these include ground stations for
downlink and targets for imaging, but `OpportunityProperties` will work with any
location added by `add_location_for_access_checking`. In these examples, 

In [ ]:
class OppPropsSatellite(sats.ImagingSatellite):
    observation_spec = [
        obs.OpportunityProperties(
            # Properties can be added by some default names
            dict(prop="priority"), 
            # They can also be normalized
            dict(prop="opportunity_open", norm=5700.0),
            # Or they can be specified by an arbitrary function
            dict(fn=lambda sat, opp: opp["r_LP_P"] + 42),
            n_ahead_observe=3,
        )
    ]
    action_spec = [act.Drift()]
    dyn_type = dyn.ImagingDynModel
    fsw_type = fsw.ImagingFSWModel

env = SatelliteTasking(
    satellite=OppPropsSatellite("OppSat-1", {}, obs_type=dict),
    scenario=scene.UniformTargets(1000),
    rewarder=data.UniqueImageReward(),
    log_level="CRITICAL",
)
observation, _ = env.reset()
observation


### Navigating the Observation

Usually, multiple observation types need to be composed to sufficiently represent the
environment for the learning agent. Simply add multiple observations to the observation
specification list to combine them in the observation.


In [ ]:
class ComposedObsSatellite(sats.Satellite):
    observation_spec = [
        obs.Eclipse(),
        obs.SatProperties(dict(prop="battery_charge_fraction"))
    ]
    action_spec = [act.Drift()]
    dyn_type = dyn.BasicDynamicsModel
    fsw_type = fsw.BasicFSWModel

env = SatelliteTasking(
    satellite=ComposedObsSatellite("PropSat-1", {}, obs_type=dict),
    log_level="CRITICAL",
)
observation, _ = env.reset()
observation


A few useful functions exist for inspecting the observation. The `observation_space`
property of the satellite and the environment return a Gym observation space to describe
the observation. In the single agent `SatelliteTasking` environment, these are the same.

<div class="alert alert-info">

**Info:** Here, we return to the `ndarray` default observation type.

</div>

In [ ]:
env = SatelliteTasking(
    satellite=ComposedObsSatellite("PropSat-1", {}),
    log_level="CRITICAL",
)
(env.observation_space, env.unwrapped.satellite.observation_space)


With the flattened-vector type observation, it can be hard for the user to relate
elements to specific observations.


In [ ]:
observation, _ = env.reset()
observation

The `observation_description` property can help the user understand what elements are 
present in the observation.

In [ ]:
env.unwrapped.satellite.observation_description


## The Action Specification

The [action specification](../api_reference/act/index.rst) works similarly to observation
specification. A list of actions is set in the class definition of the satellite.

In [ ]:
class ActionSatellite(sats.Satellite):
    observation_spec = [obs.Time()]
    action_spec = [
        # If action duration is not set, the environment max_step_duration will be used;
        # however, being explicit is always preferable
        act.Charge(duration=120.0),
        act.Desat(duration=60.0),
        # One action can be included multiple time, if different settings are desired
        act.Charge(duration=600.0,),
    ]
    dyn_type = dyn.BasicDynamicsModel
    fsw_type = fsw.BasicFSWModel

env = SatelliteTasking(
    satellite=ActionSatellite("ActSat-1", {}, obs_type=dict),
    log_level="INFO",
)
env.reset()

# Try each action; index corresponds to the order of addition
_ =env.step(0)
_ =env.step(1)
_ =env.step(2)

As with the observations, properties exist to help understand the actions available.

In [ ]:
env.action_space

In [ ]:
env.unwrapped.satellite.action_description

Some actions take additional configurations, add multiple actions to the satellite, and/or
have "special" features that are useful for manually interacting with the environment. 
For example, the imaging action can add an arbitrary number of actions corresponding to
upcoming targets and process the name of a target directly instead of operating by
action index.

In [ ]:
class ImageActSatellite(sats.ImagingSatellite):
    observation_spec = [obs.Time()]
    action_spec = [
        # Set the number of upcoming targets to consider
        act.Image(n_ahead_image=3)
    ]
    dyn_type = dyn.ImagingDynModel
    fsw_type = fsw.ImagingFSWModel

env = SatelliteTasking(
    satellite=ImageActSatellite("ActSat-2", {}),
    scenario=scene.UniformTargets(1000),
    rewarder=data.UniqueImageReward(),
    log_level="INFO",
)
env.reset()

env.unwrapped.satellite.action_description

Demonstrating the action overload feature, we task the satellite based on target name.
While this is not part of the official Gym API, we find it useful in certain cases.

In [ ]:
target = env.unwrapped.satellite.find_next_opportunities(n=10)[9]["object"]
_ = env.step(target)